# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [1]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: error during connect: Head "http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/_ping": open //./pipe/dockerDesktopLinuxEngine: The system cannot find the file specified.
See 'docker run --help'.


In [ ]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: error during connect: Head "http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/_ping": open //./pipe/dockerDesktopLinuxEngine: The system cannot find the file specified.
See 'docker run --help'.


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [3]:
!curl localhost

#the curl command should fail on our system 

/bin/bash: line 1: curl: command not found


Lets run it on the docker container 

In [5]:
!docker exec -it nginx /bin/sh -c "curl google.com"



curl: (6) Could not resolve host: google.com


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [6]:
!docker network


Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [7]:
!docker network ls 

NETWORK ID     NAME          DRIVER    SCOPE
d2b08c95b890   bridge        bridge    local
d873972cad3b   host          host      local
f7b0737ec118   none          null      local
431d092aa909   yourNetwork   bridge    local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [8]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "f7b0737ec1188b800684be7edfaea4ba5d2871bc12f00933bf55712a110ee27d",
        "Created": "2025-04-18T12:24:26.478234728-07:00",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "7ab5623c69ce176ed0f73d36588631a6fc267a86f1a39efc73f2d6c99ee7b65f": {
                "Name": "nginx",
                "EndpointID": "72fbd83858fe9381c57f262223b13a06205afe16e9c74188b46f26b497fb7278",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [25]:
!pip install docker

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 832.3 kB/s eta 0:00:00a 0:00:01


In [9]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 26.1.3


Lets create a quick code to list the containers using the docker client

In [16]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

nginx - running


Here you can see the running container and no surprise it is the nginx we started before


In [17]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [18]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

8a648f7475c54d63f2171fa48084d09f15762e8ff4ddc49d50ede0d1f4afce17


## The following needs to be run in the terminal

In [19]:
!docker run -d --name adminer -p 8080:8080 adminer


577a20c1f115ed5f764647038e5e061a5d185e91331c7d5776fd5699bb62d7f3


Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [20]:
!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


216-            "GlobalIPv6PrefixLen": 0,
217:            "IPAddress": "172.17.0.2",
218-            "IPPrefixLen": 16,
--
229-                    "Gateway": "172.17.0.1",
230:                    "IPAddress": "172.17.0.2",
231-                    "IPPrefixLen": 16,
------------------------------
mysql in network 'bridge' has IP: 172.17.0.2


In [21]:
!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


220-            "GlobalIPv6PrefixLen": 0,
221:            "IPAddress": "172.17.0.3",
222-            "IPPrefixLen": 16,
--
233-                    "Gateway": "172.17.0.1",
234:                    "IPAddress": "172.17.0.3",
235-                    "IPPrefixLen": 16,


------------------------------
adminer in network 'bridge' has IP: 172.17.0.3


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [22]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()        
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


adminer - running
mysql - running
nginx - running
Removing adminer
Removing mysql
Removing nginx


## Setting up the network

In [23]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

Error response from daemon: network with name yourNetwork already exists


In [24]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "431d092aa909f50dab25f1d2dc5eef9c97b516314e2bb79519d8198541843e8c",
        "Created": "2025-04-18T14:36:24.265817696-07:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [25]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


588d61f6039c47d4534996ef550aee1f971f77b40caa963b81d968693f0a4f91


In [26]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

5d67952613a70dcd53daef2e504cfb4aba4aa1a1af9c5584de6ac7ab29a10ad8


In [27]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [29]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,


                    "IPAddress": "172.18.0.2",
------------------------------
mysql in network 'yourNetwork' has IP: 172.18.0.2


In [27]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "IPAddress": "172.18.0.3",
PING 172.18.0.3 (172.18.0.3) 56(84) bytes of data.
64 bytes from 172.18.0.3: icmp_seq=1 ttl=64 time=0.063 ms

--- 172.18.0.3 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.063/0.063/0.063/0.000 ms
------------------------------


NameError: name 'show_container_ip' is not defined

# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 10.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 




## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/

